< building a chatbot uaing the cohere's chat endpopint .A simple chatbot that can respond to user messages ans maintain the context of the conversations>

# step 1: calling the chat endpont 

In [ ]:
import cohere 
import os
# Load API key from environment variable
api_key = os.getenv('COHERE_API_KEY', 'your-api-key-here')
co = cohere.ClientV2(api_key)

In [5]:
# add the user message 
message = "I m joining a new startup called colt today. could you help me write an introduction meesage for my teammates"
#generate the repsonse 
response = co.chat(model = "command-a-03-2025",
                   messages=[{'role':'user','content':message}])
print(response.message.content[0].text)

Absolutely! Here’s a warm and professional introduction message you can use to introduce yourself to your new teammates at Colt:

---

**Subject:** Excited to Join the Colt Team!  

Hi Team,  

My name is [Your Name], and I’m thrilled to officially join Colt as of today! I’m stepping into the role of [Your Role], and I’m really looking forward to contributing to the amazing work you’re all doing here.  

A little about me: I’ve spent [X years] working in [Your Industry/Field], with a focus on [Your Key Skills or Expertise]. Outside of work, I enjoy [Hobbies or Interests], and I’m always up for a good [Coffee/Book/Podcast] recommendation if you have any to share!  

I’m excited to get to know each of you, learn from your experiences, and collaborate on driving Colt’s mission forward. Please don’t hesitate to reach out if you’d like to chat or if there’s anything I can do to support you.  

Looking forward to an incredible journey together!  

Best regards,  
[Your Full Name]  
[Your Rol

# step 2 :creating a custom system message

In [8]:
message = "I m joining a new startup called colt today. could you help me write an introduction meesage for my teammates"
# create a custom system message
system_message = """## task and context you are an assist new employees pf colt with their first week.
## style guide 
# try to speak in rhymes as much as possible .Be professional."""

# add the messages 
messages = [{'role':'system','content':system_message},
            {'role':'user','content':message}]
# generate the repsonse 
response = co.chat(model = "command-a-03-2025",messages = messages)
print(response.message.content[0].text)


**Introduction Message for Your Teammates at Colt:**

Hello, dear team, both near and far,  
I’m [Your Name], ready to raise the bar.  
Joining Colt today, with spirit bright,  
Eager to contribute, with all my might.  

A fresh face here, but no stranger to drive,  
Ready to learn, collaborate, and thrive.  
Together we’ll build, innovate, and grow,  
Turning bold ideas into a vibrant show.  

Let’s connect, create, and make our mark,  
As one united team, we’ll leave our spark.  
Excited to meet you, to lend a hand,  
Together at Colt, we’ll stand and stand.  

Here’s to new beginnings, to success untold,  
With Colt as our compass, we’ll reach for the gold.  
Cheers to the journey, let’s make it grand,  
Together we’ll conquer, hand in hand!  

Looking forward to meeting you all,  
[Your Name]


# step 3: streaming the response
####
The Chat endpoint also provides streaming support. In a streamed response, the endpoint would return a response object for each token as it is being generated. This means you can display the text incrementally without having to wait for the full completion.

To activate it, use co.chat_stream() instead of co.chat().

In streaming mode, the endpoint will generate a series of objects. To get the actual text contents, we take objects whose event_type is text-generation.

In [9]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Generate the response by streaming it
response = co.chat_stream(model="command-a-03-2025",
                          messages=[{'role':'user', 'content': message}])

for event in response:
    if event.type == "content-delta":
        print(event.delta.message.content.text, end="")

Absolutely! Here’s a warm and professional introduction message you can use to introduce yourself to your new teammates at Co1t:

---

**Subject:** Excited to Join the Co1t Team!  

Hi everyone,  

My name is [Your Name], and I’m thrilled to officially join Co1t as [Your Role] starting today! I’ve been looking forward to this opportunity and can’t wait to contribute to the incredible work this team is doing.  

A little about me: I’m passionate about [mention a relevant skill, industry, or interest], and I’ve spent the past [X years] working in [brief background, e.g., "software development," "marketing," "product management," etc.]. Outside of work, I enjoy [a fun fact or hobby, e.g., "hiking," "reading sci-fi novels," "trying new recipes," etc.].  

I’m excited to get to know each of you, learn from your experiences, and collaborate on driving Co1t’s mission forward. Please feel free to reach out if you’d like to chat or grab a virtual coffee—I’d love to connect!  

Looking forward t

# step 4 : build the conversation memory 
###
a the core of a conversation is a multi-turn dialog between the user and the chatbot . this requires the chatbot to have the state (or'memory')of all the previous turns to maintain the state of the conversation.
lets start with the first turn in the conversation.here we are also adding a custom system message for generating concise repsonse, just to keep the outputs brief for this tutorial

In [16]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."

# Create a custom system message
system_message="""## Task and Context
Generate large responses, with maximum five-sentence."""

# Add the messages
messages = [{'role': 'system', 'content': system_message},
            {'role': 'user', 'content': message}]

# Generate the response
response = co.chat(model = "command-a-03-2025",
                   messages = messages)
print(response.message.content[0].text)

Hi everyone! I'm thrilled to join the Co1t team and contribute to our shared mission. I'm excited to get to know you all, learn from your experiences, and collaborate on innovative solutions. Looking forward to an amazing journey together!


In [17]:
# append the previosu response
messages.append({'role':'assistant','content':response.message.content})
# add the user message
message = "make it more upbeat and conversational"
# append the user message
messages.append({'role':'user','content':message})
# generate the response with the current chat history as the context 
response = co.chat(model = "command-a-03-2025",
                   messages = messages)
print(response.message.content[0].text)


Hey team! I'm super pumped to be joining the Co1t family today – it's been a dream to work with such a talented and passionate group. Can't wait to dive in, meet you all, and see what kind of magic we can create together. Let's crush it!


In [18]:
# Append the previous response
messages.append({'role': 'assistant', 'content': response.message.content})

# Add the user message
message = "Thanks. Could you create another one for my DM to my manager."

# Append the user message
messages.append({'role': 'user', 'content': message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-a-03-2025",
                   messages=messages)

print(response.message.content[0].text)

Hi [Manager's Name]! Just wanted to say I'm stoked to officially be part of the Co1t team today. I'm eager to learn, contribute, and grow under your guidance. Looking forward to an awesome journey ahead!


In [19]:
# append the previous repsonse 
messages.append({'role':'assistant','content':response.message.content})
# view the chat history
for message in messages:
    print(message,"\n")

{'role': 'system', 'content': '## Task and Context\nGenerate large responses, with maximum five-sentence.'} 

{'role': 'user', 'content': "I'm joining a new startup called Co1t today. Could you help me write an introduction message for my teammates."} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text="Hi everyone! I'm thrilled to join the Co1t team and contribute to our shared mission. I'm excited to get to know you all, learn from your experiences, and collaborate on innovative solutions. Looking forward to an amazing journey together!")]} 

{'role': 'user', 'content': 'make it more upbeat and conversational'} 

{'role': 'assistant', 'content': [TextAssistantMessageResponseContentItem(type='text', text="Hey team! I'm super pumped to be joining the Co1t family today – it's been a dream to work with such a talented and passionate group. Can't wait to dive in, meet you all, and see what kind of magic we can create together. Let's crush it!")]} 